In [ ]:
import boto3
s3 = boto3.resource('s3')
client = boto3.client('s3')
bucketName = 'revealdata-argentina'

In [ ]:
bucket = s3.Bucket(bucketName)

objs = bucket.objects.all()

files = []
for obj in objs:
    if ('metadata' not in obj.key) and ('data-subset' in obj.key):
        files.append(obj.key)

In [ ]:
metadata = []
for obj in objs:
    if ('metadata' in obj.key) and ('data-subset' in obj.key):
        metadata.append(obj.key)

In [ ]:
import pandas as pd

csv_obj = client.get_object(Bucket=bucketName, Key=files[10])
body = csv_obj['Body']
csv_string = body.read().decode('utf-8')

In [ ]:
df = pd.DataFrame(csv_string)